#### Stacks and Queues:PROPERTIES:



##### Agenda



1.  Stacks
    -   &#x2026; for delimiter pairing
    -   &#x2026; for postfix expression evaluation
    -   &#x2026; for tracking execution and *backtracking*

2.  Queues
    -   &#x2026; for tracking execution and *backtracking*
    -   &#x2026; for fair scheduling (aka "round-robin" scheduling)
    -   &#x2026; for apportioning work

3.  Run-time analysis



##### Overview



While the list data structure is incredibly useful, both implementations
we explored (array-backed and linked) have operations that run in $O(N)$
time, which make them non-ideal for use with large, growing collections
of data.

By further restricting the list API, however &#x2014; in particular, by
*isolating points of access to either the front or end of the data set*
&#x2014; we can create data structures whose operations are all $O(1)$, and
remain very useful in their own right.



##### 1.  Stacks



Stacks are linear data structures which only permit access to one "end"
of the data collection. We can only append ("push") items onto the tail
end (a.k.a. the "top") of a stack, and only the most recently added item
can be removed ("popped"). The last item to be pushed onto a stack is
therefore the first one to be popped off, which is why we refer to
stacks as **last-in, first out** (**LIFO**) structures.



###### Array-based implementation



In [1]:
# array-backed implementation

class Stack:
    def __init__(self):
        self.data = []

    def push(self, val): # O(1) - append is O(1)
        self.data.append(val)

    def pop(self): # O(1) - deleting at the end is O(1)
        assert not self.empty()
        val = self.data[-1]
        del self.data[-1]
        return val

    def peek(self):
        assert not self.empty()
        return self.data[-1]

    def empty(self):
        return self.data == []

    def __bool__(self):
        return not self.empty()

    def __repr__(self):
        return self.data.__repr__()

    def __str__(self):
        return self.__repr__()

# Out[191]:

In [1]:
s = Stack()
for x in range(10):
    s.push(x)
s

# Out[192]:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [1]:
while s:
    print(s.pop())

9
8
7
6
5
4
3
2
1
0

###### Linked list implementation



In [1]:
# linked implementation

class LStack:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next

    def __init__(self):
        self.top = None

    def push(self, val):
        lc = self.Node(val,next=self.top)
        self.top = lc

    def pop(self):
        assert not self.empty()
        val = self.top.val
        self.top = self.top.next
        return val

    def peek(self):
        assert not self.empty()
        return self.top.val

    def empty(self):
        return not self.top

    def __bool__(self):
        return not self.empty()

    def __iter__(self):
        l = self.top
        while l:
            yield l
            l = l.next

    def __repr__(self):
        return '[' + ','.join([ str(l.val) for l in self ]) + ']'

    def __str__(self):
        return self.__repr__()

# Out[194]:

In [1]:
sl = LStack()
for x in range(10):
    sl.push(x)
sl

# Out[195]:
[9,8,7,6,5,4,3,2,1,0]

In [1]:
while sl:
    print(sl.pop())

9
8
7
6
5
4
3
2
1
0

###### &#x2026; for delimiter pairing



Stacks are used by parsers to decide if delimited expressions are
well-formed.

-   e.g., `'(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))'` - is valid
-   `'((1 + 2)'` - not valid, because first parenthesis is not closed
-   `'(1 + 2)))'` - not valid, because parenthesis are closed that were not opened
-   `'(1 + 1)'` - valid
-   `'(1 + 1) + ((1 + (2 / 4)))'` -



In [1]:
def check_parens(expr):
    s = Stack()
    for c in expr:
        if c == '(':
            s.push(c)
        if c == ')':
            if s.empty():
                return False
            s.pop()
    return s.empty()

# Out[197]:

In [1]:
check_parens('()')

# Out[198]:
True

In [1]:
check_parens('((()))')

# Out[199]:
True

In [1]:
check_parens('()(()()(()))')

# Out[200]:
True

In [1]:
check_parens('(')

# Out[201]:
False

In [1]:
check_parens('())')

# Out[202]:
False

In [1]:
check_parens('(1 + 2 * (3 - (4 / 2) + 5) - (6 + 1))')

# Out[203]:
True

###### &#x2026; for postfix expression (aka "reverse polish notation")



evaluation
Stacks are used for the evaluation of postfix arithmetic expressions
(which can be easily converted back and forth between the more common
infix expressions).

e.g., `'(1 + 2) * 5'` $\rightarrow$ `'1 2 + 5 *'`



In [1]:
def is_int(s):
    try:
        i = int(s)
    except ValueError:
        return False
    return True

def apply_op(op,left,right):
    if op == '+':
        return left + right
    if op == '*':
        return left * right
    if op == '-':
        return left - right
    if op == '/':
        return left / right
    if op == '//':
        return left // right
    raise Exception(f"cannot handle operator {op}")

def eval_postfix(expr):
    s = Stack()
    toks = expr.split()
    for t in toks:
        if is_int(t):
            s.push(int(t))
        else:
            val = apply_op(t, s.pop(), s.pop())
            s.push(val)
    return s.pop()

# Out[204]:

In [1]:
eval_postfix('1 2 + 5 *')

# Out[205]:
15

-   `((5 * 2) + 1)` - this is 11



In [1]:
eval_postfix('1 2 5 * +')

# Out[206]:
11

In [1]:
# ((1 + 2) * (3 + 2)) * 10 = 150
eval_postfix('1 2 + 3 2 + * 10 *')

# Out[207]:
150

###### &#x2026; for tracking execution and backtracking



In [1]:
maze_str = """######
I    #
# ## #
# ####
#    O
######"""

def parse_maze(maze_str):
    '''Parses a string representing a maze into a 2D array.'''
    grid = []
    for line in maze_str.split('\n'):
        grid.append(['# IO'.index(c) for c in line.strip()])
    return grid

def print_maze(grid):
    '''Takes a 2D array maze representation and pretty-prints it.
         The contents of the 2D maze are in the range 0-5, which are interpreted as:

    0: a wall
    1: an unvisited (i.e., not previously traversed) path
    2: the maze entrance
    3: the maze exit
    4: a discovered but unvisited path
    5: a visited path
    '''
    for r in grid:
        print(''.join('# IO!+'[c] for c in r))

# Out[208]:

In [1]:
parse_maze(maze_str)

# Out[209]:
#+BEGIN_EXAMPLE
  [[0, 0, 0, 0, 0, 0],
  [2, 1, 1, 1, 1, 0],
  [0, 1, 0, 0, 1, 0],
  [0, 1, 0, 0, 0, 0],
  [0, 1, 1, 1, 1, 3],
  [0, 0, 0, 0, 0, 0]]
#+END_EXAMPLE

In [1]:
print_maze(parse_maze(maze_str))

######
I    #
# ## #
# ####
#    O
######

In [1]:
maze = parse_maze(maze_str)
maze[1][0] = maze[1][1] = 5
maze[1][2] = maze[2][1] = 4
print_maze(maze)

######
++!  #
#!## #
# ####
#    O
######

In [1]:
class Move:
    '''Represents a move in the maze between orthogonally adjacent locations
        `frm` and `to`, which are both (row,col) tuples.'''
    def __init__(self, frm, to):
        self.frm = frm
        self.to  = to

    def __repr__(self):
        return '({},{}) -> ({},{})'.format(self.frm[0], self.frm[1],
                                           self.to[0],  self.to[1])

def moves(maze, loc):
    '''Returns all possible moves within a maze from the provide location.'''
    moves = [Move(loc, (loc[0]+d[0], loc[1]+d[1]))
             for d in ((-1, 0), (1, 0), (0, -1), (0, 1))
             if loc[0]+d[0] in range(len(maze)) and
             loc[1]+d[1] in range(len(maze[0])) and
             maze[loc[0]+d[0]][loc[1]+d[1]] in (1, 2, 3)]
    return moves

# Out[212]:

In [1]:
maze = parse_maze(maze_str)
print_maze(maze)

######
I    #
# ## #
# ####
#    O
######

In [1]:
moves(maze, (1, 0))

# Out[214]:
[(1,0) -> (1,1)]

In [1]:
moves(maze, (1, 1))

# Out[215]:
[(1,1) -> (2,1), (1,1) -> (1,0), (1,1) -> (1,2)]

In [1]:
maze[1][0] = 5
moves(maze, (1, 1))

# Out[216]:
[(1,1) -> (2,1), (1,1) -> (1,2)]

In [1]:
from time import sleep
from IPython.display import clear_output

def mark(maze, loc):
    '''Marks a loc in the maze as having been discovered'''
    if maze[loc[0]][loc[1]] != 3:
        maze[loc[0]][loc[1]] = 4

def visit(maze, loc):
    '''Marks a loc in the maze as having been visited'''
    maze[loc[0]][loc[1]] = 5

def display(maze):
    '''Prints out the maze after clearing the cell -- useful for animation.'''
    clear_output(True)
    print_maze(maze)
    #sleep(0.1)

# Out[217]:

In [1]:
def solve_maze(maze, entry):
    '''Searches for the exit in a maze starting from the given entry point.

         The algorithm works as follows:

         1. Visit the entry point and save all possible moves from that location.
         2. Remove and consider one of the saved moves. If it is the exit, we are done,
            otherwise visit the destination and save all possible moves from there.
         3. If we run out of saved moves, we can't find an exit.

         When we save a move, we also mark it as "discovered" in the maze.

         The data structure used to save moves plays a critical role in how maze
         exploration proceeds!
    '''
    for m in moves(maze, entry):
        save_move(m)
        visit(maze, entry)
        while not out_of_moves():
            move = next_move()
            if maze[move.to[0]][move.to[1]] == 3:
                break
            display(maze)
            visit(maze, move.to)
            for m in moves(maze, move.to):
                mark(maze, m.to)
                save_move(m)
                display(maze)

# Out[218]:

In [1]:
move_stack = LStack()

def save_move(move):
    move_stack.push(move)

def next_move():
    return move_stack.pop()

def out_of_moves():
    return move_stack.empty()

# Out[219]:

In [1]:
move_stack = LStack()
maze_str = """######
              I    #
              # ## #
              # ####
              #    O
              ######"""
solve_maze(parse_maze(maze_str), (1, 0))

# Out[221]:

In [1]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

# Out[54]:

In [1]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

Intuitively, because the stack is a last-in, first-out data structure,
it keeps following moves down the most recently discovered path until it
either reaches the exit or reaches a dead end. It then picks up from the
previously discovered path. We call this type of exploration
*depth-first traversal*.



##### 1.  Queues



###### What are queues?



Queues are linear data structures wherein we are only permitted to
 append ("enqueue") items onto the rear, and remove ("dequeue") items
 from the front. The oldest item still in a queue is therefore the next
 one to be dequeued, which is why we refer to a queue as a first-in,
 first-out (FIFO) structure. It is helpful to think of a queue as being
 the model for a line at a typical supermarket checkout aisle (first
 customer in, first customer to be checked out).



###### Array-backed implementation



In [1]:
# array-backed implementation

class Queue:
    def __init__(self):
        self.data = []

    def enqueue(self, val):
        pass

    def dequeue(self):
        assert not self.empty()
        pass

    def empty(self):
        pass

    def __bool__(self):
        return not self.empty()

In [1]:
q = Queue()
for x in range(10):
    q.enqueue(x)

In [1]:
while q:
    print(q.dequeue())

###### Circular array-backed implementation



In [1]:
# circular array-backed implementation
class Queue:

    def __init__(self, size):
        self.data = [None] * size
        self.head = self.tail = -1

    def enqueue(self, val):
        self.tail = (self.tail + 1) % len(self.data)
        if self.tail == self.head:
            raise Exception("queue is full")
        self.data[self.tail] = val

    def dequeue(self):
        assert not self.empty()
        self.head += 1
        return self.data[self.head]

    def empty(self):
        return self.head == self.tail

    def markel(self, i):
        s = str(self.data[i])
        if i == self.head:
            s = '*' + s
        if i == self.tail:
            s = '&' + s
        return s

    def __repr__(self):
        return '[' + ','.join([ self.markel(i) for i in range(0, len(self.data)) ]) + ']'

    def __str__(self):
        return self.__repr__()

:results:
 # Out[180]:
 :end:

In [1]:
q = Queue(10)
for x in range(6):
    q.enqueue(x)
q

:results:
 # Out[186]:
 : [0,1,2,3,4,&5,None,None,None,None]
 :end:

In [1]:
for x in range(5):
    print(q.dequeue())

:results:
 0
 1
 2
 3
 4
 :end:

In [1]:
q

:results:
 # Out[188]:
 : [0,1,2,3,*4,&5,None,None,None,None]
 :end:

In [1]:
for x in range(6, 12):
    q.enqueue(x)
q

:results:
 # Out[189]:
 : [10,&11,2,3,*4,5,6,7,8,9]
 :end:

###### Single linked list with tail implementation



In [1]:
# linked implementation

class Queue:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next

    def __init__(self):
        self.head = self.tail = None

    def enqueue(self, val):
        n = Queue.Node(val, None)
        if self.tail:
            self.tail.next = n
        else:
            self.head = n
        self.tail = n

    def dequeue(self):
        assert not self.empty()
        n = self.head
        self.head = self.head.next
        if n == self.tail:
            self.tail = None
        return n.val

    def empty(self):
        return not self.head

    def __bool__(self):
        return not self.empty()

    def __iter__(self):
        n = self.head
        while n:
            yield n.val
            n = n.next

    def __repr__(self):
        return "[" + ','.join([ str(v) for v in self ]) + ']'

:results:
 # Out[190]:
 :end:

In [1]:
q = Queue()
for x in range(10):
    q.enqueue(x)
q

:results:
 # Out[67]:
 : [0,1,2,3,4,5,6,7,8,9]
 :end:

In [1]:
while q:
    print(q.dequeue())

:results:
 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 :end:

###### &#x2026; for tracking execution and backtracking



In [1]:
move_queue = Queue()

def save_move(move):
    move_queue.enqueue(move)

def next_move():
    return move_queue.dequeue()

def out_of_moves():
    return move_queue.empty()

# Out[71]:

In [1]:
maze_str = """######
              I    #
              # ## #
              # ####
              #    O
              ######"""

solve_maze(parse_maze(maze_str), (1, 0))

# Out[72]:

In [1]:
maze_str = """#################
              I #       #     #
              # ##### # # # # #
              #     # # # # # #
              # ### ### # # ###
              #   #       #   O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

In [1]:
maze_str = """#################
              I               #
              # # # # # # # # #
              # # # # # # # # #
              # ###############
              #               O
              #################"""

solve_maze(parse_maze(maze_str), (1, 0))

Intuitively, because the queue is a first-in, first-out &#x2013; i.e., *fair*
&#x2013; data structure, it keeps rotating through all the paths which haven't
yet dead-ended, making just one move further down each time. We call
this type of exploration **breadth-first traversal**.

Are there types of mazes which might be more suitably tackled using one approach over the other (i.e., depth-first vs. breadth-first)?



###### &#x2026; for fair scheduling (aka "round-robin" scheduling)



Queues are often used to either carry out or simulate the fair
allocation of resources. Here we implement a "round-robin" scheduler for
permitting different tasks to run for small, fixed periods of time until
they complete:



In [1]:
from random import randint
from time import sleep

task_queue = Queue()
for i in range(3):
    task_queue.enqueue(('Job {}'.format(i), randint(3, 6)))

# Out[73]:

In [1]:
n = task_queue.head
while n:
    print(n.val)
    n = n.next

('Job 0', 5)
('Job 1', 6)
('Job 2', 4)

In [1]:
while not task_queue.empty():
    job, time_left = task_queue.dequeue()
    print('Running', job)
    sleep(0.1)
    time_left -= 1
    if time_left > 0:
        print('Re-queueuing', job, 'with remaining time =', time_left)
        task_queue.enqueue((job, time_left))
    else:
        print('*', job, 'done')

# Out[76]:

In [1]:
from random import randint
from time import sleep

task_stack = Stack()
for i in range(3):
    task_stack.push(('Job {}'.format(i), randint(3, 6)))

# Out[85]:

In [1]:
while not task_stack.empty():
    job, time_left = task_stack.pop()
    print('Running', job)
    sleep(0.1)
    time_left -= 1
    if time_left > 0:
        print('Re-stackuing', job, 'with remaining time =', time_left)
        task_stack.push((job, time_left))
    else:
        print(' *', job, 'done')

Running Job 2
Re-stackuing Job 2 with remaining time = 3
Running Job 2
Re-stackuing Job 2 with remaining time = 2
Running Job 2
Re-stackuing Job 2 with remaining time = 1
Running Job 2
 * Job 2 done
Running Job 1
Re-stackuing Job 1 with remaining time = 4
Running Job 1
Re-stackuing Job 1 with remaining time = 3
Running Job 1
Re-stackuing Job 1 with remaining time = 2
Running Job 1
Re-stackuing Job 1 with remaining time = 1
Running Job 1
 * Job 1 done
Running Job 0
Re-stackuing Job 0 with remaining time = 3
Running Job 0
Re-stackuing Job 0 with remaining time = 2
Running Job 0
Re-stackuing Job 0 with remaining time = 1
Running Job 0
 * Job 0 done

###### &#x2026; for doling out work



Queues are also frequently used as a sort of conveyer belt for multiple
worker functions to draw from. Here we implement a "work queue" pattern
used by multiple threads of execution:

    from threading import Thread, Lock
    from time import sleep
    import random
    
    lock = Lock()
    def worker_fn(cid, q):
        while True:
            try:
                with lock:
                    work = q.dequeue()
            except: # queue is empty
                sleep(1)
                continue
            if work == 'Stop':
                print('Consumer', cid, 'stopping.')
                break
            else:
                print('Consumer', cid, 'processing', work)
                sleep(random.random())
    
    work_queue = Queue()
    for i in range(5):
        Thread(target=worker_fn, args=(i, work_queue)).start()

    import threading
    threading.active_count()

    10

    for i in range(10):
        with lock:
            work_queue.enqueue(i)

    for i in range(5):
        with lock:
            work_queue.enqueue('Stop')



##### 1.  Run-time analysis



Stack & Queue implementations:

-   **Insertion** (push and enqueue) = $O(1)$
-   **Deletion** (pop and dequeue) = $O(1)$

